Notebook for testing the general noisy system

## PSR

In [1]:
from qibochem.measurement.result import expectation_from_samples


def evaluate(circuit, hamiltonian, nshots):
    if nshots is None:
        return hamiltonian.expectation(circuit().state())
    return expectation_from_samples(circuit, hamiltonian, nshots, group_pauli_terms="qwc")

In [3]:
def parameter_shift(
    circuit,
    hamiltonian,
    parameter_index,
    initial_state=None,
    scale_factor=1,
    nshots=None,
    nruns=1,
    repeated = False
):
    """In this method the parameter shift rule (PSR) is implemented.
    Given a circuit U and an observable H, the PSR allows to calculate the derivative
    of the expected value of H on the final state with respect to a variational
    parameter of the circuit.
    There is also the possibility of setting a scale factor. It is useful when a
    circuit's parameter is obtained by combination of a variational
    parameter and an external object, such as a training variable in a Quantum
    Machine Learning problem. For example, performing a re-uploading strategy
    to embed some data into a circuit, we apply to the quantum state rotations
    whose angles are in the form: theta' = theta * x, where theta is a variational
    parameter and x an input variable. The PSR allows to calculate the derivative
    with respect of theta' but, if we want to optimize a system with respect its
    variational parameters we need to "free" this procedure from the x depencency.
    If the `scale_factor` is not provided, it is set equal to one and doesn't
    affect the calculation.
    Args:
        circuit (:class:`qibo.models.circuit.Circuit`): custom quantum circuit.
        hamiltonian (:class:`qibo.hamiltonians.Hamiltonian`): target observable.
        parameter_index (int): the index which identifies the target parameter in the circuit.get_parameters() list
        initial_state ((2**nqubits) vector): initial state on which the circuit acts (default None).
        scale_factor (float): parameter scale factor (default None).
        repeated (bool): indicates if the gates that use the parameter have a translational symmetry
    Returns:
        np.float value of the derivative of the expectation value of the hamiltonian
        with respect to the target variational parameter.
    Example:
        .. testcode::
            import qibo
            import numpy as np
            from qibo import hamiltonians, gates
            from qibo.models import Circuit
            from qibo.derivative import parameter_shift
            # defining an observable
            def hamiltonian(nqubits = 1):
                m0 = (1/nqubits)*hamiltonians.Z(nqubits).matrix
                ham = hamiltonians.Hamiltonian(nqubits, m0)
                return ham
            # defining a dummy circuit
            def circuit(nqubits = 1):
                c = Circuit(nqubits = 1)
                c.add(gates.RY(q = 0, theta = 0))
                c.add(gates.RX(q = 0, theta = 0))
                c.add(gates.M(0))
                return c
            # initializing the circuit
            c = circuit(nqubits = 1)
            # some parameters
            test_params = np.random.randn(2)
            c.set_parameters(test_params)
            test_hamiltonian = hamiltonian()
            # running the psr with respect to the two parameters
            grad_0 = parameter_shift(circuit = c, hamiltonian = test_hamiltonian, parameter_index = 0)
            grad_1 = parameter_shift(circuit = c, hamiltonian = test_hamiltonian, parameter_index = 1)
    """
    """We modify the Qibo implementation to allow multiple parameter_index. For the case where a parameter is reused in different gates"""
    gradient = 0
    try:
        iter(parameter_index) # check if it is iterable
    except:
        parameter_index = [parameter_index]
    if repeated:
        nparam = len(parameter_index)
        parameter_index = parameter_index[:1]
    for parameter_index in parameter_index:
        # inheriting hamiltonian's backend
        backend = hamiltonian.backend

        # getting the gate's type
        gate = circuit.associate_gates_with_parameters()[parameter_index]

        # getting the generator_eigenvalue
        ###try: generator_eigenval = gate.generator_eigenvalue()
        ###except: 
        #generator_eigenval = 0.5   #### CAN'T DO SOMETHING LIKE THIS... ERROR GETS LOGGED ANYWAYS BY QIBO FUNCTION RAISE_ERROR, EVEN IF CAUGHT BY TRY
        # getting the generator_eigenvalue
        generator_eigenval = gate.generator_eigenvalue()

        # defining the shift according to the psr
        s = np.pi / (4 * generator_eigenval)

        # saving original parameters and making a copy
        original = np.asarray(circuit.get_parameters()).copy()
        shifted = original.copy()

        # forward shift and evaluation
        shifted[parameter_index] += s
        circuit.set_parameters(shifted)

        forward = 0
        backward = 0


        if nshots == None:
            forward = hamiltonian.expectation(
                backend.execute_circuit(circuit=circuit, initial_state=initial_state).state()
            )

            # backward shift and evaluation
            shifted[parameter_index] -= 2 * s
            circuit.set_parameters(shifted)

            backward = hamiltonian.expectation(
                backend.execute_circuit(circuit=circuit, initial_state=initial_state).state()
            )

        else:
            
            copied = shifted.copy()

            for _ in range(nruns):

                forward += evaluate(circuit, hamiltonian, nshots) #np.float64(hamiltonian.expectation_from_circuit(circuit, nshots=nshots).real)
                """backend.execute_circuit(
                    circuit=circuit, 
                    initial_state=initial_state, 
                    nshots=nshots
                    ).expectation_from_samples(hamiltonian)"""

                # backward shift and evaluation
                shifted[parameter_index] -= 2 * s
                circuit.set_parameters(shifted)

                backward += evaluate(circuit, hamiltonian, nshots)
                """backend.execute_circuit(
                    circuit=circuit, 
                    initial_state=initial_state, 
                    nshots=nshots
                    ).expectation_from_samples(hamiltonian)  """

                # restoring the original circuit
                shifted = copied.copy()
                circuit.set_parameters(copied)

            forward /= nruns
            backward /= nruns
                
        circuit.set_parameters(original)
        
        gradient += generator_eigenval * (forward - backward) * scale_factor

    if repeated: gradient *= nparam
    return gradient

## Non PSR

In [31]:
import qibo
from qibo import gates, Circuit
from qibo.symbols import I, X, Y, Z
import numpy as np

# Hamiltonian for the TFIsing model
def hamiltonianTFI(N, g):
    ham = I(0)*0
    for q in range(N):
        ham += Z(q)*Z((q+1)%N) + g * X(q)
    return qibo.hamiltonians.SymbolicHamiltonian(-ham)

qibo.gates.RZZ.generator_eigenvalue = lambda self: 0.5
def circ_inv(N, p):
    params = np.random.random((2*p,))*4*np.pi
    circuit_inv = Circuit(N)
    for q in range(N):
        circuit_inv.add(gates.H(q))  # WE START WITH |+> state
    for itp in range(p):
        for itzz in range(N):
            circuit_inv.add(gates.RZZ(itzz, (itzz+1)%N, params[2*itp]))
        for itx in range(N):
            circuit_inv.add(gates.RX(itx, params[2*itp + 1]))
    return circuit_inv, params

def circ_noinv(N, p):
    params2 = np.random.random((3*p,))*4*np.pi
    circuit_noinv = Circuit(N)
    for q in range(N):
        circuit_noinv.add(gates.H(q))
    for itp in range(p):
        for itzz in range(N):
            circuit_noinv.add(gates.RZZ(itzz, (itzz+1)%N, params2[3*itp]))
        for itx in range(N):
            circuit_noinv.add(gates.RX(itx, params2[3*itp + 1]))
        for ity in range(N):
            circuit_noinv.add(gates.RY(ity, params2[3*itp + 2]))
    return circuit_noinv, params2
    
def circs_shots_noisy_jac(N, p, g = 1, custom_operator=None, nshots=400, noise_map=None, epsilon = 0.02, noise=True):
    if noise_map is None:
        noise_map = {i: list(zip(["X", "Z"], [epsilon, epsilon])) for i in range(N)}

    if custom_operator is None: 
        hamiltonian = hamiltonianTFI(N, g)
    else:
        hamiltonian = custom_operator
    
    circuit_inv, params = circ_inv(N,p)
    circuit_noinv, params2 = circ_noinv(N,p)

    if noise:
        circuit_inv = circuit_inv.with_pauli_noise(noise_map)
        circuit_noinv = circuit_noinv.with_pauli_noise(noise_map)

    def loss(params):
        circuit_inv.set_parameters(np.repeat(params,N))
        return evaluate(circuit_inv, hamiltonian, nshots)#np.float64(hamiltonian.expectation_from_circuit(circuit_inv, nshots=nshots).real)

    def loss2(params):
        circuit_noinv.set_parameters(np.repeat(params,N))
        return evaluate(circuit_noinv, hamiltonian, nshots)#np.float64(hamiltonian.expectation_from_circuit(circuit_noinv, nshots=nshots).real)

    gate_dependence_inv = {i: list(range(i*N, (i+1)*N)) for i in range(0,2*p)}
    gate_dependence_noinv = {i: list(range(i*N, (i+1)*N)) for i in range(0,3*p)}
    def jac_psr_inv(x, *args):
        circuit_inv.set_parameters(np.repeat(x,N))
        return np.array([parameter_shift(circuit_inv, hamiltonian, gate_dependence_inv[param], nshots=nshots, nruns=1, repeated=True) for param in range(len(params))])
    
    def jac_psr_noinv(x, *args):
        circuit_noinv.set_parameters(np.repeat(x,N))
        return np.array([parameter_shift(circuit_noinv, hamiltonian, gate_dependence_noinv[param], nshots=nshots, nruns=1, repeated=True) for param in range(len(params2))])
    
    return circuit_inv, circuit_noinv, params, params2, loss, loss2, jac_psr_inv, jac_psr_noinv


## With noise, some optimizers don't behave as well. For instance BFGS

# We tried implementing a noise resiliant alternative but it does not seem to work too well, because it stops convergence up to random error
# But we know we can keep running Gradient Descent, and stochastically it will converge


# We implement here a simple gradient descent, with a simple learning rate hyperparameter
def optimizer_gds(parameters, loss, gradient, Nepochs=250, lr=1e-2, epochs_print = 50, hyper={}, return_gradients=False):
    loss_list = np.empty((Nepochs,))
    if return_gradients: grad_list = np.empty((Nepochs,len(parameters)))
    for epoch in range(Nepochs):
        grad_val = gradient(parameters)
        parameters = parameters - lr * grad_val
        
        loss_val = loss(parameters)
        if not(epochs_print is None):
            if epoch % epochs_print == 0 or epoch == Nepochs - 1:
                print(f"Epoch {epoch}, Loss: {loss_val}")

        loss_list[epoch] = loss_val
        if return_gradients: grad_list[epoch] = grad_val
    
    if return_gradients: return parameters, loss_list, grad_list
    return parameters, loss_list

In [28]:
c1 = Circuit(4)
c1.add(gates.H(q) for q in range(4))
c1.add(gates.Z(q) for q in range(4))
c1.add(gates.M(q) for q in range(4))

print(c1.with_pauli_noise({q: [('X', 0.00000)] for q in range(4)}))#{q: [('X', 0)]} for q in range(4)))

0: ─H─Z─M─
1: ─H─Z─M─
2: ─H─Z─M─
3: ─H─Z─M─


In [34]:
N = 6
nshots = 5
Nepochs = 40
p = 3
lr = 0.4e-2
rep = 0

epsilon = 0.02

np.random.seed(2000)
init_params = np.random.random((2*p,))*4*np.pi
#print(init_params)


data = []
for epsilon in [0.00000, 0.00001, 0.0001, 0.001, 0.01]:#, 0.001, 0.01, 0.1]:
    for nshots in [40, 400]:#, 400, 4000, 40000, 4000000, None]:
        for noise in [True]:
            print(f"epsilon: {epsilon:.5f}, nshots: {nshots if not(nshots is None) else 0: 6d}, rep: {rep: 2d}")
            #np.random.seed(2025)
            circuit_inv, circuit_noinv, params, params2, loss, loss2, jac1, jac2 = circs_shots_noisy_jac(N, p, nshots=nshots, noise=noise, epsilon=epsilon)
            #print(params)
            params = init_params.copy()
            #circuit_inv.draw()
            _, loss_list, grads = optimizer_gds(params, loss, jac1, Nepochs = Nepochs, epochs_print=None, lr=lr, return_gradients=True)
            data.append({"nshots": nshots, "epsilon":epsilon, "loss": loss_list, "gradients": grads, "noise": noise})
            


epsilon: 0.00000, nshots:     40, rep:  0
epsilon: 0.00000, nshots:    400, rep:  0
epsilon: 0.00001, nshots:     40, rep:  0
epsilon: 0.00001, nshots:    400, rep:  0
epsilon: 0.00010, nshots:     40, rep:  0
epsilon: 0.00010, nshots:    400, rep:  0
epsilon: 0.00100, nshots:     40, rep:  0
epsilon: 0.00100, nshots:    400, rep:  0


In [7]:
print(data)

[{'nshots': 40, 'epsilon': 0.0001, 'loss': array([-1.85, -1.8 , -1.15, -1.85, -0.95, -2.3 , -1.3 , -2.3 , -1.7 ,
       -1.75, -2.25, -2.7 , -2.8 , -2.2 , -2.75, -3.15, -2.1 , -2.55,
       -2.35, -3.5 , -2.7 , -2.25, -2.95, -3.6 , -2.9 ]), 'gradients': array([[-3.00000000e-01, -1.80000000e+00, -4.65000000e+00,
         2.55000000e+00,  4.65000000e+00,  3.45000000e+00],
       [ 6.00000000e+00,  1.20000000e+00, -2.10000000e+00,
         1.05000000e+00,  4.80000000e+00,  6.00000000e-01],
       [ 5.85000000e+00, -3.15000000e+00, -3.30000000e+00,
        -1.50000000e-01, -1.05000000e+00,  4.35000000e+00],
       [ 1.95000000e+00,  1.33226763e-15,  7.50000000e-01,
         3.30000000e+00,  2.55000000e+00, -1.20000000e+00],
       [ 4.20000000e+00, -1.80000000e+00,  1.65000000e+00,
        -1.05000000e+00,  1.65000000e+00, -1.50000000e+00],
       [ 5.70000000e+00,  6.00000000e-01, -1.80000000e+00,
        -2.10000000e+00,  2.40000000e+00,  4.50000000e-01],
       [ 2.25000000e+00,  1.2000

In [71]:
grads

array([[-6.  ,  3.12,  0.24,  1.44],
       [-5.52, -1.92,  2.88, -1.2 ],
       [-2.4 ,  2.16,  0.24, -3.84],
       [-2.16, -0.72,  3.36,  4.56]])

In [47]:
%matplotlib macosx
import matplotlib.pyplot as plt
Nepochs = 40
#plt.ion()
for el in data:
    nshots = el["nshots"]
    #rep = el["rep"]
    epsilon = el["epsilon"]
    epoch_range = np.arange(Nepochs)
    error = np.sqrt(2*N/nshots) if nshots is not None else 0
    loss_list = el["loss"]
    grads = el["gradients"]
    noise = el["noise"]
    plt.title(f"N: {N}, p: {p}\nnshots: {nshots}, noise: {noise}")#, epsilon: {epsilon}")#, rep: {rep}, epsilon: {epsilon}")
    plt.ylabel("Loss Function")
    plt.xlabel("Epoch")
    plt.plot(epoch_range, loss_list)
    plt.fill_between(epoch_range, loss_list+error, loss_list-error, alpha=0.1)
    #plt.plot([0,Nepochs-1],[min(loss_list)]*2, '--c')
    plt.plot([0,Nepochs-1],[-7.72740661031255]*2, ':')
    ax_grads = plt.twinx()
    ax_grads.set_ylabel("Gradients")
    max_grads = np.average(grads, axis=1)
    ax_grads.plot(epoch_range, max_grads, '-.y')
    ax_grads.fill_between(epoch_range, max_grads+error/np.sqrt(2), max_grads-error/np.sqrt(2), alpha=0.1)

    plt.show(block=True)


# MISSING:
#   ALSO PLOT WITH axis.twinx or something like that GRADIENTS AND HOW NOISE COMPARES TO THEM
    

    

In [12]:
# BAD DATA WITH NOISE 25epochs
#data2 = data.copy()

In [69]:
# DATA WITHOUT ERROR NOISE 50epochs
#dataclean = data.copy()

In [109]:
# Data with noise, but not too god data 40epochs
#datanoise = data.copy()

In [44]:
# DAta wihout error noise 40epochs
#dataclean2 = data.copy()

NameError: name 'dataclean2' is not defined

## TESTING

In [50]:
import qibo
circ = qibo.Circuit(3)
circ.add(qibo.gates.X(0))

circ2 = qibo.Circuit(3)
circ2.add(qibo.gates.H(0))

circ += circ2

circ.draw()

0: ─X─H─
1: ─────
2: ─────


In [49]:
from qibo import Circuit, gates
from qibo.config import raise_error
from typing import List
import math
import numpy as np

In [199]:
def dicke_state(nqubits: int, weight: int, all_to_all: bool = False, **kwargs):
    """Create a circuit that prepares the Dicke state :math:`\\ket{D_{k}^{n}}`.

    The Dicke state :math:`\\ket{D_{k}^{n}}` is the equal superposition of all :math:`n`-qubit
    computational basis states with fixed-Hamming-weight :math:`k`.
    The circuit prepares the state deterministically with :math:`O(k \\, n)` gates and :math:`O(n)` depth,
    or :math:`O(k \\log\\frac{n}{k})` depth under the assumption of ``all-to-all`` connectivity.

    Args:
        nqubits (int): number of qubits :math:`n`.
        weight (int): Hamming weight :math:`k` of the Dicke state.
        all_to_all (bool, optional): If ``False``, uses implementation from Ref. [1].
            If ``True``, uses shorter-depth implementation from Ref. [2].
            Defaults to ``False``.
        kwargs: Additional arguments for Circuit initialization.

    Returns:
        :class:`qibo.models.circuit.Circuit` : Circuit that prepares :math:`\\ket{D_{k}^{n}}`.

    References:
        1. Andreas Bärtschi and Stephan Eidenbenz, *Deterministic preparation of Dicke states*,
        `22nd International Symposium on Fundamentals of Computation Theory, FCT'19, 126-139  (2019)
        <https://doi.org/10.1007/978-3-030-25027-0_9>`_.
        2. Andreas Bärtschi and Stephan Eidenbenz, *Short-Depth Circuits for Dicke State Preparation*,
        `IEEE International Conference on Quantum Computing & Engineering (QCE), 87--96 (2022)
        <https://doi.org/10.1109/QCE53715.2022.00027>`_.
    """
    if weight < 0 or weight > nqubits:
        raise_error(
            ValueError, f"weight must be between 0 and {nqubits}, but got {weight}."
        )

    circuit = Circuit(nqubits, **kwargs)

    if weight == 0:
        return circuit

    if not all_to_all:
        # Start with |0⟩^(n-k) |1⟩^k
        circuit.add(gates.X(qubit) for qubit in range(nqubits - weight, nqubits))

        _add_dicke_unitary_gate(circuit, range(nqubits), weight)

        return circuit

    # We prepare disjoint sets of qubits
    disjoint_sets = [
        {
            "qubits": list(range(weight * it, weight * (it + 1))),
            "tier": weight,
            "children": [],
        }
        for it in range(nqubits // weight)
    ]
    nmodk = nqubits % weight
    if nmodk != 0:
        disjoint_sets.append(
            {
                "qubits": list(range(nqubits - nmodk, nqubits)),
                "tier": nmodk,
                "children": [],
            }
        )
    # reverse to have in ascending order of tier
    disjoint_sets = list(reversed(disjoint_sets))

    trees = disjoint_sets.copy()
    # combine lowest tier trees into one tree
    while len(trees) > 1:
        first_smallest = trees.pop(0)
        second_smallest = trees.pop(0)
        second_smallest["tier"] += first_smallest["tier"]
        second_smallest["children"].append(first_smallest)
        new = second_smallest
        # put new combined tree in list mantaining ordering
        trees.insert(sum(x["tier"] < new["tier"] for x in trees), new)

    root = trees[0]
    # We initialize |0>^(n-k)|1>^k  bitstring at root qubits
    circuit.add(gates.X(q) for q in root["qubits"][-weight:])

    # undo the union-by-tier algorithm:
    # split each root's (x) highest tier child y, and add WBD acting on both
    while len(trees) < len(disjoint_sets):
        cut_nodes = []
        for node in trees:
            if len(node["children"]) > 0:
                # find highest tier child
                y = max(node["children"], key=lambda x: x["tier"])
                # add WBD acting on both sets of qubits
                _add_wbd_gate(
                    circuit,
                    node["qubits"],
                    y["qubits"],
                    node["tier"],
                    y["tier"],
                    weight,
                )
                # cut tree splitting x and y
                node["tier"] -= y["tier"]
                node["children"].remove(y)
                cut_nodes.append(y)
        trees += cut_nodes

    for node in disjoint_sets:
        _add_dicke_unitary_gate(
            circuit, node["qubits"], min(weight, len(node["qubits"]))
        )

    return circuit


def _add_scs_gate(circuit: Circuit, qubits: List[int], weight: int):
    """In-place addition of a Split & Cyclic Shift (SCS) gate to ``circuit``.
    Implements the SCS_{n,k} unitary from Definition 3 of the paper [1].
    Acts on the last weight+1 qubits of the nqubits passed qubits.
    """
    if weight == 0:
        return  # SCS_{n,0} is identity

    nqubits = len(qubits)
    last_qubit = qubits[-1]  # qubits[nqubits - 1]
    target_qubit = qubits[-2]
    # first_qubit = qubits[nqubits - 1 - weight]

    # Gate (i) - acts on last two qubits
    theta = 2 * np.arccos(np.sqrt(1 / nqubits))
    circuit.add(gates.CNOT(target_qubit, last_qubit))
    circuit.add(gates.RY(target_qubit, theta).controlled_by(last_qubit))
    circuit.add(gates.CNOT(target_qubit, last_qubit))

    # Gates (ii)_ℓ for ℓ from 2 to k
    for l in range(2, weight + 1):
        theta = 2 * np.arccos(np.sqrt(l / nqubits))
        target_qubit = qubits[-(l + 1)]
        control_qubit = qubits[-l]

        # Implement the three-qubit gate (ii)_ℓ
        circuit.add(gates.CNOT(target_qubit, last_qubit))
        circuit.add(
            gates.RY(target_qubit, theta).controlled_by(control_qubit, last_qubit)
        )
        circuit.add(gates.CNOT(target_qubit, last_qubit))


def _add_dicke_unitary_gate(circuit: Circuit, qubits: List[int], weight: int):
    """In-place addition to ``circuit`` of a U_{n,k} gate from Definition 2 of the paper [1]."""
    nqubits = len(qubits)
    for m in range(nqubits, weight, -1):
        # Add SCS_{m,k} acting on last k+1 qubits
        _add_scs_gate(circuit, qubits[:m], weight)

    # Recursively build the unitary U_n,k
    for m in range(weight, 0, -1):
        # Add SCS_{m,m-1} acting on last m qubits
        _add_scs_gate(circuit, qubits[:m], m - 1)


def _add_wbd_gate(
    circuit: Circuit,
    first_register: List[int],
    second_register: List[int],
    nqubits: int,
    mqubits: int,
    weight: int,
):
    """In-place addition of a Weight Distribution Block (WBD) to ``circuit``.
    Implements the :math:`WBD^{n,m}_k` unitary from Definition 2 of the paper [2].
    Only acts on first_register and second_register, last k qubits
    Our circuit is mirrored, as paper [2] uses a top-bottom circuit <-> right-left bitstring convention
    """

    if mqubits > nqubits / 2:
        raise_error(ValueError, "``m`` must not be greater than ``n - m``.")

    # only acts on last k qubits
    first_register = first_register[-weight:]
    second_register = second_register[-weight:]

    # if m>k, m is truncated. Operations involving the most significant k-m digits can be removed

    # (1) Switching from unary encoding to one hot encoding
    circuit.add(gates.CNOT(q, q + 1) for q in first_register[-2::-1])

    # (2) Adding a supperposition of hamming weights into the second register
    # this can be optimized
    # We follow Figure 4, but adjust definition of xi and si (suffix sum) to match
    theta_gate = lambda qubit, theta: gates.RY(qubit, 2 * math.acos(theta))
    for l in range(weight, 0, -1):
        x = [
            math.comb(mqubits, i) * math.comb(nqubits - mqubits, l - i)
            for i in range(l)
        ]
        s = math.comb(nqubits, l)
        circuit.add(
            theta_gate(second_register[-1], math.sqrt(x[0] / s)).controlled_by(
                first_register[-l]
            )
        )
        s -= x[0]
        for qubit in range(1, min(l, mqubits)):
            circuit.add(
                theta_gate(
                    second_register[-qubit - 1], math.sqrt(x[qubit] / s)
                ).controlled_by(first_register[-l], second_register[-qubit])
            )
            s -= x[qubit]

    # (3) Go back to unary encoding, undoing one hot encoding
    circuit.add(gates.CNOT(q, q + 1) for q in first_register[:-1])

    # (4) Substracting weight i in the first register from weight l in the second register.
    # fredkin, controlled swaps (decomposed into CNOT and Toffoli)
    fredkin = lambda control, s1, s2: (
        gates.CNOT(s2, s1),
        gates.TOFFOLI(control, s1, s2),
        gates.CNOT(s2, s1),
    )

    dif = max(0, weight - mqubits)
    for control in range(dif, weight):
        for q in range(control):
            circuit.add(
                fredkin(
                    second_register[control - dif],
                    first_register[control - q],
                    first_register[control - q - 1],
                )
            )
        circuit.add(gates.CNOT(second_register[control - dif], first_register[0]))



In [234]:
backend = qibo.get_backend()

def test_wbd_gate(backend, nqubits, mqubits, weight, density_matrix):
    wbd_circ = Circuit(nqubits, density_matrix=density_matrix)
    first_register = list(range(mqubits, nqubits))
    second_register = list(range(mqubits))
    if mqubits > nqubits-mqubits:
        with pytest.raises(ValueError):
            _add_wbd_gate(wbd_circ, first_register, second_register, nqubits, mqubits, weight)
    else:
        _add_wbd_gate(wbd_circ, first_register, second_register, nqubits, mqubits, weight)

        # Build expected WBD state vector (Definition 2 from [2])
        # Test for all considered (0 <= l <= weight) input states: |0>^n-l |1>^l
        for l in range(weight+1):
            initial = np.zeros(2**(nqubits), dtype=complex)
            target = np.zeros(2**(nqubits), dtype=complex)

            # initial state |0>^n-l |1>^l
            initial[2**l-1] = 1
            
            comb_n_l = math.comb(nqubits, l)
            for i in range(min(l,mqubits)+1):
                index = 2**(nqubits-mqubits) * (2**(i) - 1) + 2**(l-i)-1
                target[index] = np.sqrt(math.comb(mqubits,i)*math.comb(nqubits-mqubits, l-i) / comb_n_l)

            initial = backend.cast(initial, dtype=target.dtype)
            target = backend.cast(target, dtype=target.dtype)

            if density_matrix:
                initial = backend.np.outer(initial, backend.np.conj(initial.T))
                target = backend.np.outer(target, backend.np.conj(target.T))

            result = backend.execute_circuit(wbd_circ, initial_state = initial)
            state = result.state()

            backend.assert_allclose(state, target)

test_wbd_gate(backend, 5, 2, 3, False)

In [129]:
c1 = Circuit(4)

c1.add(gates.X(0))

print(c1().state())

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [129]:
def dicke_state(nqubits: int, k: int, all_to_all_connectivity: bool = False, **kwargs):
    """Generate an :math:`n`-qubit Dicke state defined as

    .. math::
        \\ket{D^n_k} = \\frac{1}{{n\\choose k}} \\sum_{\\ket{b}\\in B} \\ket{b}

    where :math:`B={\\ket{b}:b\\in {0,1}}^{\\otimes n} \\text{and } |b|=k}` is the set of
    bitstrings with fixed Hamming weight k (i.e. number of non-zero bits).
    Implemented following Lemma 2 from [1]`arXiv:1904.07358 <https://arxiv.org/abs/1904.07358>`_ with
    :math:`O(kn)` two-qubit gates and depth :math:`O(n)`. Additionally, if all-to-all connectivity is
    allowed, the improved version from [2]`arXiv:2207.09998 <https://arxiv.org/abs/2207.09998>`_ with depth
    :math:`O(k\\log(\\frac{n}{k}))` is used instead.

    Args:
        nqubits (int): number of qubits :math:`n >= 2`.
        k (int): bitstring's Hamming weight k.
        all_to_all_connectivity (bool): Decides which implementation to use.
        kwargs (dict, optional): additional arguments used to initialize a Circuit object.
            For details, see the documentation of :class:`qibo.models.circuit.Circuit`.

    Returns:
        :class:`qibo.models.circuit.Circuit`: Circuit that prepares the Dicke state.

    TEST:
    ### TEST FOR VALIDATING WBD/U BY REPLICATING QUIRK CIRCUIT AFTER DEFINITION 2 (IMPLEMENT FIGURE 2) from ref [2]
    print("MANUAL WBD CONSTRUCTION TEST: ")
    nqubits = 11
    circ = Circuit(nqubits)
    qubits = range(nqubits)

    # flip last k bits, to generate Dnk
    k = 3
    circ.add( gates.X(nqubits-1-q) for q in range(k) )   # can test for l=0, 1, 2 or 3

    circ += _dicke_WBD(nqubits, qubits, 11, 5, 3)

    circ += _dicke_WBD(nqubits, qubits[-6:], 6, 3, 3)
    circ += _dicke_WBD(nqubits, qubits[:5], 5, 2, 3)

    circ += _dicke_u(nqubits, qubits[:2], 2)
    circ += _dicke_u(nqubits, qubits[2:5], 3)
    circ += _dicke_u(nqubits, qubits[5:8], 3)
    circ += _dicke_u(nqubits, qubits[8:], 3)

    circ.draw()
    print(circ())


    ### WE ALSO VALIDATE IT GIVES THE SAME AS WITHOUT FULL_CONENCTIVITY
    print("\n\nNON-FULL CONNECTIVITY SANITY CHECK: ")
    print(dicke_state(nqubits, k)())


    ### AND FINALLY TEST WE ALSO GET THIS CIRCUIT FROM THE ALL_TO_ALL_CONNECTIVITY
    print("\n\nALL-TO-ALL CONNECTIVITY REPLICATE: ")
    circ2 = dicke_state(nqubits, k, all_to_all_connectivity=True)
    circ2.draw()
    print(circ2())
    """
    circuit = Circuit(nqubits, **kwargs)
    if k == nqubits:
        circuit.add(gates.X(q) for q in range(nqubits))
        return circuit
    if all_to_all_connectivity:
        ## confusing because big endian interpretation in the paper for Unk is reversed from paper for all-to-all
        ## Circuit drawings are inverted with respect to paper for us

        # 1. We prepare disjoint sets of qubits / trees. Tier in ascending order
        disjoint_sets = [
            {"qubits": list(range(k * it, k * (it + 1))), "tier": k, "children": []}
            for it in range(nqubits // k)
        ]
        nmodk = nqubits % k
        if nmodk != 0:
            disjoint_sets.append(
                {
                    "qubits": list(range(nqubits - nmodk, nqubits)),
                    "tier": nmodk,
                    "children": [],
                }
            )
        disjoint_sets = list(
            reversed(disjoint_sets)
        )  # we reverse to have in ascending order of tier

        trees = disjoint_sets.copy()
        # combine lowest tier trees into one tree. Could do binary search but isn't bottleneck
        while len(trees) > 1:
            trees[1]["tier"] += trees[0]["tier"]
            trees[1]["children"].append(trees[0])
            root = trees[1]
            del trees[1]
            del trees[0]
            trees.insert(sum(x["tier"] < root["tier"] for x in trees), root)

        # We initialize [1]xk  bitstring we distribute with WBD
        circuit.add(gates.X(q) for q in trees[0]["qubits"][-k:])

        # undo the union-by-tier algorithm applying WDB gates with root x and highest tier child y (y<x by construct.)
        it = 0
        while it < len(trees):
            node = trees[it]
            if len(node["children"]) > 0:
                y = max(node["children"], key=lambda x: x["tier"])

                circuit += _dicke_WBD(
                    nqubits,
                    None,
                    node["tier"],
                    y["tier"],
                    k,
                    second_register=node["qubits"],
                    first_register=y["qubits"],
                    **kwargs,
                )

                node["tier"] -= y["tier"]
                node["children"].remove(y)
                trees.append(y)
                if len(node["children"]) == 0:
                    it += 1
            else:
                it += 1

        for node in disjoint_sets:
            circuit += _dicke_u(
                nqubits, node["qubits"], min(k, len(node["qubits"])), **kwargs
            )

    else:
        circuit.add(gates.X(q) for q in range(nqubits - k, nqubits))
        circuit += _dicke_u(nqubits, range(nqubits), k, **kwargs)

    return circuit


def _dicke_u(nqubits: int, qubits: List[int], k: int, **kwargs):
    """Implements :math:`U_{n,k}` gate from `arXiv:1904.07358 <https://arxiv.org/abs/1904.07358>`_"""
    circuit = Circuit(nqubits, **kwargs)
    n = len(qubits)
    if k == 0:
        # State already in dicke state (as there is only 1 bitstring with hamming weight k=0 and k=n)
        return circuit
    for l in range(n, k, -1):
        circuit += _dicke_scs(nqubits, qubits[:l], k, **kwargs)
    for l in range(k, 1, -1):
        circuit += _dicke_scs(nqubits, qubits[:l], l - 1, **kwargs)
    return circuit


def _dicke_scs(nqubits: int, qubits: List[int], k: int, **kwargs):
    """Implements ``nqubits`` :math:`SCS_{n,k}` gate from `arXiv:1904.07358 <https://arxiv.org/abs/1904.07358>`_"""
    n = len(qubits)
    circuit = Circuit(nqubits, **kwargs)
    circuit.add(gates.CNOT(qubits[n - 2], qubits[n - 1]))
    circuit.add(
        gates.RY(qubits[n - 2], 2 * math.acos(math.sqrt(1 / n))).controlled_by(
            qubits[n - 1]
        )
    )
    circuit.add(gates.CNOT(qubits[n - 2], qubits[n - 1]))
    for l in range(2, k + 1):
        circuit.add(gates.CNOT(qubits[n - 1 - l], qubits[n - 1]))
        circuit.add(
            gates.RY(qubits[n - 1 - l], 2 * math.acos(math.sqrt(l / n))).controlled_by(
                qubits[n - 1], qubits[n - l]
            )
        )
        circuit.add(gates.CNOT(qubits[n - 1 - l], qubits[n - 1]))
    return circuit


def _dicke_WBD(
    nqubits: int,
    qubits: List[int],
    n: int,
    m: int,
    k: int,
    first_register: List[int] = None,
    second_register: List[int] = None,
    **kwargs,
):
    """Implements :math:`WBD^{n,m}_k` gate from `arXiv:2207.09998 <https://arxiv.org/abs/2207.09998>`_
    Only acts relevantly on last k digits of first m digits and last k digits of all n digits.
    These relevant first and second registers can be passed directly"""
    ## A bit confusing because they changed big endian interpretation in the paper!!!
    # we asume n-m > m (m <= n/2)
    if m > n / 2:
        raise (ValueError("m must not be greater than n-m"))

    circuit = Circuit(nqubits, **kwargs)

    # IMPORTANT REMINDER, our first register and second register are reverse as paper,
    # Our drawn circuits are mirrored to circuit.
    # if m>k, m is truncated. Operations involving the most significant k-m digits can be removed
    if (first_register is None) and (second_register is None):
        first_register = qubits[max(m - k, 0) : m]
        second_register = qubits[n - k : n]

    # (1) Switching from unary encoding to one hot encoding
    circuit.add(gates.CNOT(q, q + 1) for q in reversed(second_register[:-1]))

    # (2) Adding a supperposition of hamming weights into the second register
    # this can be optimized
    # We follow Figure 4, but adjust definition of xi and si (suffix sum) to match
    for l in reversed(range(1, k + 1)):
        x = [math.comb(m, i) * math.comb(n - m, l - i) for i in range(l)]
        s = math.comb(n, l)
        circuit.add(
            gates.RY(
                first_register[-1], 2 * math.acos(math.sqrt(x[0] / s))
            ).controlled_by(second_register[-l])
        )
        s -= x[0]
        for q in range(1, min(l, m)):
            circuit.add(
                gates.RY(
                    first_register[-q - 1], 2 * math.acos(math.sqrt(x[q] / s))
                ).controlled_by(second_register[-l], first_register[-q])
            )
            s -= x[q]

    # (3) Go back to unary encoding, undoing one hot encoding
    circuit.add(gates.CNOT(q, q + 1) for q in second_register[:-1])

    # (4) Substracting weight i in the first register from weight l in the second register.
    # We use fredkin, controlled swap (can be decomposed into CNOT and Toffoli)
    fredkin = lambda control, s1, s2: (
        gates.CNOT(s2, s1),
        gates.TOFFOLI(control, s1, s2),
        gates.CNOT(s2, s1),
    )

    dif = max(0, k - m)
    for control in range(dif, k):
        for q in range(control):
            circuit.add(
                fredkin(
                    first_register[control - dif],
                    second_register[control - q],
                    second_register[control - q - 1],
                )
            )
        circuit.add(gates.CNOT(first_register[control - dif], second_register[0]))

    return circuit


In [179]:
circ = dicke_state(4,1, all_to_all=True)

circ.draw()

print(circ())

0: ───RY─o──────o──X─
1: ───|──|──────RY─o─
2: ─X─o──X─o──X──────
3: ────────RY─o──────
(0.5+0j)|0001> + (0.5+0j)|0010> + (0.5+0j)|0100> + (0.5+0j)|1000>


In [116]:
### TEST FOR CHECKING WBD44
nq = 8
circ = Circuit(nq)
q = range(nq)

_dicke_WBD(nq, q, nq, 4, 4).draw()

NameError: name '_dicke_WBD' is not defined

In [122]:
### TEST FOR VALIDATING WBD/U BY REPLICATING QUIRK CIRCUIT AFTER DEFINITION 2 (IMPLEMENT FIGURE 2)
print("MANUAL WBD CONSTRUCTION TEST: ")
nqubits = 11
circ = Circuit(nqubits)
qubits = range(nqubits)

# flip last k bits, to generate Dnk
k = 3
circ.add( gates.X(nqubits-1-q) for q in range(k) )   # can test for l=0, 1, 2 or 3

_add_wbd_gate(circ, qubits[-6:], qubits[:5], 11, 5, 3)

_add_wbd_gate(circ, qubits[-3:], qubits[-6:-3], 6, 3, 3)
_add_wbd_gate(circ, qubits[2:5], qubits[:2], 5, 2, 3)

_add_dicke_unitary_gate(circ, qubits[8:], 3)
_add_dicke_unitary_gate(circ, qubits[5:8], 3)
_add_dicke_unitary_gate(circ, qubits[2:5], 3)
_add_dicke_unitary_gate(circ, qubits[:2], 2)

circ.draw()
print(circ())


### WE ALSO VALIDATE IT GIVES THE SAME AS WITHOUT FULL_CONENCTIVITY
print("\n\nNON-FULL CONNECTIVITY SANITY CHECK: ")
print(dicke_state(nqubits, k)())


### AND FINALLY TEST WE ALSO GET THIS CIRCUIT FROM THE ALL_TO_ALL_CONNECTIVITY UP TO RESHUFLING OF PARALLELIZABLE BLOCKS
print("\n\nALL-TO-ALL CONNECTIVITY REPLICATE: ")
circ2 = dicke_state(nqubits, k, all_to_all=True)
circ2.draw()
print(circ2())


MANUAL WBD CONSTRUCTION TEST: 
0 :     ─────────────────────────────────────────o──────────────────────────── ...
1 :     ─────────────────────────────────────────|───o───o──────────────────── ...
2 :     ───────────────────RY────────────────────|───|───|───o─────o───o────── ...
3 :     ────────────────RY─o─────RY──────────────|───|───|───|─────|───|────── ...
4 :     ─────────────RY─o──|──RY─o──RY───────────|───|───|───|─────|───|────── ...
5 :     ───────────o─|──|──|──|──|──|──o─────────X─o─X─o─X───|───o─X─o─X────── ...
6 :     ─────────o─X─|──|──|──|──|──|──X─o─────────X─o─X───o─X─o─X─o─X──────── ...
7 :     ───────o─X───|──|──|──|──|──|────X─o───────────────X─o─X─────────────R ...
8 :     ─X───o─X─────o──o──o──|──|──|──────X─o─────────────────────────────o─o ...
9 :     ─X─o─X────────────────o──o──|────────X─o─────────────────────────o─X── ...
10:     ─X─X────────────────────────o──────────X─────────────────────────X──── ...

0 : ... ───────────────────────────────────────────────

## Old tests 

In [135]:
### TEST FOR VALIDATING WBD/U BY REPLICATING QUIRK CIRCUIT AFTER DEFINITION 2 (IMPLEMENT FIGURE 2)
print("MANUAL WBD CONSTRUCTION TEST: ")
nqubits = 11
circ = Circuit(nqubits)
qubits = range(nqubits)

# flip last k bits, to generate Dnk
k = 3
circ.add( gates.X(nqubits-1-q) for q in range(k) )   # can test for l=0, 1, 2 or 3

circ += _dicke_WBD(nqubits, qubits, 11, 5, 3)

circ += _dicke_WBD(nqubits, qubits[-6:], 6, 3, 3)
circ += _dicke_WBD(nqubits, qubits[:5], 5, 2, 3)

circ += _dicke_u(nqubits, qubits[:2], 2)
circ += _dicke_u(nqubits, qubits[2:5], 3)
circ += _dicke_u(nqubits, qubits[5:8], 3)
circ += _dicke_u(nqubits, qubits[8:], 3)

circ.draw()
print(circ())


### WE ALSO VALIDATE IT GIVES THE SAME AS WITHOUT FULL_CONENCTIVITY
print("\n\nNON-FULL CONNECTIVITY SANITY CHECK: ")
print(dicke_state(nqubits, k)())


### AND FINALLY TEST WE ALSO GET THIS CIRCUIT FROM THE ALL_TO_ALL_CONNECTIVITY
print("\n\nALL-TO-ALL CONNECTIVITY REPLICATE: ")
circ2 = dicke_state(nqubits, k, all_to_all_connectivity=True)
circ2.draw()
print(circ2())


MANUAL WBD CONSTRUCTION TEST: 
0 :     ────────────────────────────────────────────────────────────────────── ...
1 :     ────────────────────────────────────────────────────────────────────── ...
2 :     ─────────────RY──────────────o──────────────────────────────────────── ...
3 :     ──────────RY─o─────RY────────|───o───o──────────────────────────────── ...
4 :     ───────RY─o──|──RY─o──RY─────|───|───|───o─────o───o────────────────── ...
5 :     ───────|──|──|──|──|──|──────|───|───|───|─────|───|───────────RY───── ...
6 :     ───────|──|──|──|──|──|──────|───|───|───|─────|───|────────RY─o─────R ...
7 :     ───────|──|──|──|──|──|──────|───|───|───|─────|───|─────RY─o──|──RY─o ...
8 :     ─X───o─o──o──o──|──|──|──o───X─o─X─o─X───|───o─X─o─X───o─o──o──o──|──| ...
9 :     ─X─o─X──────────o──o──|──X─o───X─o─X───o─X─o─X─o─X───o─X──────────o──o ...
10:     ─X─X──────────────────o────X───────────X─o─X─────────X──────────────── ...

0 : ... ────────────────────────────────────────RY────R

In [107]:
list(np.count_nonzero(dicke_state(15, k, all_to_all=True)().state()) for k in range(16))

[1,
 15,
 105,
 455,
 1365,
 3003,
 5005,
 6435,
 6435,
 5005,
 3003,
 1365,
 455,
 105,
 15,
 1]

In [86]:
import math
import numpy as np
x = lambda n, m, l: [math.comb(m,i)*math.comb(n-m,l-i) for i in range(l+1)]
s = lambda n, m, l: [sum(x(n,m,l)[i:]) for i in range(l)]

print(x(5,2,3))
print(s(5,2,3))

np.array([0,1,2,3])[-4]

[1, 6, 3, 0]
[10, 9, 3]


0

In [377]:
math.comb(6,3)

20

In [9]:
dicke_state(162, 18, all_to_all_connectivity=True)

In [10]:
l = [1, 2, 5, 7]

for x in l:
    print(x)
    if x == 2:
        l.m
        del x

l


1
2


AttributeError: 'list' object has no attribute 'm'